# Fake news classifier using LSTM

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
df.shape

(20800, 5)

In [6]:
df=df.dropna()

In [7]:
## Get the independent feature
x = df.drop('label',axis=1)

In [8]:
y=df['label']

In [9]:
x.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [10]:
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [11]:
print(x.shape
,y.shape)

(18285, 4) (18285,)


In [12]:
import tensorflow as tf

In [13]:
tf.__version__

'2.12.0-rc1'

In [14]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [15]:
### Vocabulary size
voc_size=5000

In [16]:
messages = x.copy()
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [17]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [18]:
messages.reset_index(inplace=True)

In [19]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [20]:
import nltk
import re
from nltk.corpus import stopwords

In [21]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bhush\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [22]:
## Data Preprocessing
from nltk.stem.porter import PorterStemmer ## Stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0,len(messages)):
    review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [23]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [24]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]

In [25]:
onehot_repr

[[3186, 15, 943, 2368, 3648, 682, 4684, 964, 2896, 3031],
 [4819, 1492, 2099, 77, 1492, 3123, 844],
 [266, 1217, 2073, 1033],
 [3930, 2373, 1142, 2962, 2141, 1968],
 [2531, 1492, 2221, 1119, 2870, 3950, 1492, 424, 4559, 3934],
 [2231,
  1531,
  4592,
  3282,
  3413,
  1143,
  3113,
  394,
  1538,
  1181,
  3847,
  4454,
  2521,
  2363,
  844],
 [3092, 2703, 1525, 4155, 1364, 2865, 1248, 3682, 2337, 1147, 2327],
 [3389, 443, 2198, 3638, 1281, 1218, 1143, 3609, 2337, 1147, 2327],
 [4927, 3147, 3433, 1363, 2718, 980, 2730, 3670, 1143, 4203],
 [98, 3286, 4013, 2051, 424, 3406, 1145, 86],
 [822, 1667, 4053, 2396, 4998, 709, 3593, 1170, 1658, 3661, 1722],
 [2962, 938, 3648, 980, 1143, 1281],
 [618, 3799, 4981, 2338, 4875, 1615, 2150, 3486, 3690],
 [2311, 407, 3721, 502, 4937, 1473, 2191, 2337, 1147, 2327],
 [1849, 3433, 1251, 3979, 337, 2337, 1147, 2327],
 [122, 4771, 1339, 4153, 4784, 2085, 1888, 3082, 3390, 3572],
 [3360, 4069, 1492],
 [2850, 1598, 667, 4705, 1143, 4874, 1786, 844],
 [1463

# Embedding Representation

In [26]:
sent_length=20
embedded_docs = pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  964 2896 3031]
 [   0    0    0 ... 1492 3123  844]
 [   0    0    0 ... 1217 2073 1033]
 ...
 [   0    0    0 ... 2337 1147 2327]
 [   0    0    0 ... 4727 1512    6]
 [   0    0    0 ... 4425 3466 2920]]


In [27]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 4819, 1492, 2099,   77, 1492, 3123,  844])

In [37]:
## Creating model
embedding_vector_features=40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100)) ## 100 neurons
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [38]:
len(embedded_docs),y.shape

(18285, (18285,))

In [39]:
import numpy as np

In [40]:
x_final=np.array(embedded_docs)
y_final = np.array(y)

In [41]:
x_final.shape,y_final.shape

((18285, 20), (18285,))

In [42]:
from sklearn.model_selection import train_test_split

In [43]:
X_train, X_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.33, random_state=42)

# Model Training

In [44]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 7s 27ms/step - loss: 0.3293 - accuracy: 0.8433 - val_loss: 0.1956 - val_accuracy: 0.9170
Epoch 2/10
192/192 [==============================] - 5s 25ms/step - loss: 0.1399 - accuracy: 0.9435 - val_loss: 0.2012 - val_accuracy: 0.9152
Epoch 3/10
192/192 [==============================] - 5s 25ms/step - loss: 0.1012 - accuracy: 0.9611 - val_loss: 0.2238 - val_accuracy: 0.9094
Epoch 4/10
192/192 [==============================] - 5s 24ms/step - loss: 0.0740 - accuracy: 0.9748 - val_loss: 0.2763 - val_accuracy: 0.9165
Epoch 5/10
192/192 [==============================] - 5s 25ms/step - loss: 0.0510 - accuracy: 0.9840 - val_loss: 0.3616 - val_accuracy: 0.9027
Epoch 6/10
192/192 [==============================] - 5s 25ms/step - loss: 0.0339 - accuracy: 0.9887 - val_loss: 0.3652 - val_accuracy: 0.9100
Epoch 7/10
192/192 [==============================] - 5s 24ms/step - loss: 0.0220 - accuracy: 0.9931 - val_loss: 0.4497 - val_accuracy: 0.9069

# Adding Dropout

In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
#embedding_vector_features=40
#model=Sequential()
#model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
#model.add(Dropout(0.3))
#model.add(LSTM(100))
#model.add(Dropout(0.3))
#model.add(Dense(1,activation='sigmoid'))
#model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])#

In [45]:
y_pred=model.predict(X_test)

189/189 [==============================] - 1s 5ms/step


In [46]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [47]:
from sklearn.metrics import confusion_matrix

In [48]:
confusion_matrix(y_test,y_pred)

array([[3134,  285],
       [ 298, 2318]], dtype=int64)

In [49]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9033968516984259

In [50]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.91      0.92      0.91      3419
           1       0.89      0.89      0.89      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035

